In [53]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [52]:
#Headers Utilisé pour BeautifulSoup
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'fr,fr-FR;q=0.8,en-US;q=0.5,en;q=0.3',
}

# Recuperation des Liens de tous les sites touristiques

On creer d'abord un array avec toutes les regions, dans chaque regions il y a plusieurs pages, donc on regarde chaque page, et on recupere les liens de chaque page, et on les ajoute dans un dictionnaire avec le nom de la region comme key et le lien comme value. Ensuite on a plus qu'a ouvrir tous les liens et recuperer les informations de chaque site touristique.


In [54]:
def get_all_pages_urls(base_url, headers, all_france_urls):

    '''
    Fonction qui prend un url et va chercher tout les liens de la page, et les ajoute dans un dictionnaire 
    avec le nom de la region comme key et le lien comme value.
    '''
    page_number = 1
    
    while True:
        # Construire l'URL de la page
        if page_number == 1:
            current_url = base_url
        else:
            current_url = base_url.replace('.htm', f'_{page_number}.htm')
        
        print(f"Traitement de la page: {current_url}")
        
        # Faire la requête
        response = requests.get(current_url, headers=headers)
        
        # Si la page n'existe pas, sortir de la boucle
        if response.status_code != 200:
            break
            
        # Parser la page
        soup = BeautifulSoup(response.content, 'html.parser')
        content_container = soup.find('section', id='content_container')
        
        if not content_container:
            break
            
        # Trouver tous les liens
        links = content_container.find_all('a')
        found_links = False
        
        for link in links:
            href = link.get('href')
            text = link.text.strip()
            
            # Vérifier si c'est un lien .htm et qu'il n'est pas déjà dans notre dictionnaire
            if href and href.endswith('.htm'):
                # Ajouter le domaine si le lien est relatif
                if not href.startswith('http'):
                    href = 'https://www.france-voyage.com' + href
                
                all_france_urls[text] = href
                found_links = True
        
        # Si aucun nouveau lien n'est trouvé, sortir de la boucle
        if not found_links:
            break
            
        page_number += 1
        time.sleep(1)  # Pause pour éviter de surcharger le serveur

In [43]:
# Liste des URLs de base pour chaque région fait a la main
base_url_region = ['https://www.france-voyage.com/tourisme/hauts-de-france-region.htm',
                   'https://www.france-voyage.com/tourisme/normandie-region.htm',
                   'https://www.france-voyage.com/tourisme/grand-est-region.htm',
                   'https://www.france-voyage.com/tourisme/ile-de-france-region.htm',
                   'https://www.france-voyage.com/tourisme/bretagne-region.htm',
                   'https://www.france-voyage.com/tourisme/pays-de-la-loire-region.htm',
                   'https://www.france-voyage.com/tourisme/centre-val-de-loire-region.htm',
                   'https://www.france-voyage.com/tourisme/bourgogne-franche-comte-region.htm',
                   'https://www.france-voyage.com/tourisme/nouvelle-aquitaine-region.htm',
                   'https://www.france-voyage.com/tourisme/auvergne-rhone-alpes-region.htm',
                   'https://www.france-voyage.com/tourisme/occitanie-region.htm',
                   'https://www.france-voyage.com/tourisme/provence-alpes-cote-d-azur-region.htm',
                   'https://www.france-voyage.com/tourisme/corse-region.htm'
                    ]

# Dictionnaire global pour tous les liens .htm
all_france_urls = {}

for base_url in base_url_region:
    print(f"\nTraitement de la région: {base_url}")
    get_all_pages_urls(base_url, headers, all_france_urls)
    time.sleep(1) 

#SAUVEGARDE DES URLS DANS UN FICHIER
with open('france_urls.txt', 'w', encoding='utf-8') as f:
    for text, url in all_france_urls.items():
        f.write(f"{text}\t{url}\n")


Traitement de la région: https://www.france-voyage.com/tourisme/hauts-de-france-region.htm
Traitement de la page: https://www.france-voyage.com/tourisme/hauts-de-france-region.htm
Traitement de la page: https://www.france-voyage.com/tourisme/hauts-de-france-region_2.htm
Traitement de la page: https://www.france-voyage.com/tourisme/hauts-de-france-region_3.htm
Traitement de la page: https://www.france-voyage.com/tourisme/hauts-de-france-region_4.htm

Traitement de la région: https://www.france-voyage.com/tourisme/normandie-region.htm
Traitement de la page: https://www.france-voyage.com/tourisme/normandie-region.htm
Traitement de la page: https://www.france-voyage.com/tourisme/normandie-region_2.htm
Traitement de la page: https://www.france-voyage.com/tourisme/normandie-region_3.htm
Traitement de la page: https://www.france-voyage.com/tourisme/normandie-region_4.htm

Traitement de la région: https://www.france-voyage.com/tourisme/grand-est-region.htm
Traitement de la page: https://www.f

# Recuperation du textes dans chaque liens possible

In [47]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time


def fix_url(url):
    """Corrige l'URL en ajoutant /tourisme/"""
    if 'france-voyage.com' in url and '/tourisme/' not in url:
        # Sépare le domaine et le reste de l'URL
        parts = url.split('france-voyage.com')
        return f"https://www.france-voyage.com/tourisme/{parts[1]}"
    return url

def get_site_info(url, headers):
    try:
        # Corriger l'URL
        fixed_url = fix_url(url)
        print(f"URL corrigée: {fixed_url}")
        
        # Faire la requête
        response = requests.get(fixed_url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Trouver le conteneur principal
        content_container = soup.find('section', id='content_container')
        
        if content_container:
            # Supprimer les publicités
            for aside in content_container.find_all('aside', class_='encart'):
                aside.decompose()
            
            # Récupérer tout le texte et le nettoyer
            text = ' '.join(p.get_text(strip=True) for p in content_container.find_all('p') if p.get_text(strip=True))
            return text
        return ""
        
    except Exception as e:
        print(f"Erreur pour l'URL {fixed_url}: {str(e)}")
        return ""


# Lire le fichier des URLs
sites_data = []
with open('france_urls.txt', 'r', encoding='utf-8') as f:
    for line in f:
        name, url = line.strip().split('\t')
        sites_data.append({'name': name, 'url': url})

# Créer le DataFrame initial
df = pd.DataFrame(sites_data)

# Ajouter la colonne de texte
print("Récupération des informations pour chaque site...")
texts = []
total_sites = len(df)

for index, row in df.iterrows():
    print(f"Traitement du site {index + 1}/{total_sites}: {row['name']}")
    text = get_site_info(row['url'], headers)
    texts.append(text)
    
    # Pause toutes les 10 requêtes
    if (index + 1) % 10 == 0:
        time.sleep(1)

df['text_info'] = texts

# Sauvegarder le DataFrame
df.to_csv('sites_touristiques.csv', index=False, encoding='utf-8')

# Afficher un aperçu
print("\nAperçu du DataFrame:")
print(df.head())
print(f"\nNombre total de sites traités: {len(df)}")
print(f"Nombre de sites avec du texte: {df['text_info'].str.len().gt(0).sum()}")

Récupération des informations pour chaque site...
Traitement du site 1/2260: Le château de Pierrefonds9.484Sur les traces d'un édifice féodal reconstruit au XIXe siècle !
URL corrigée: https://www.france-voyage.com/tourisme/chateau-pierrefonds-572.htm
Traitement du site 2/2260: Gerberoy9.171Symbolisé par la rose, venez vivre une expérience sensorielle unique dans ce village bourré de charme !
URL corrigée: https://www.france-voyage.com/tourisme/gerberoy-571.htm
Traitement du site 3/2260: La Côte d'Opale8.474Toute la splendeur du littoral du nord-ouest de la France
URL corrigée: https://www.france-voyage.com/tourisme/cote-opale-626.htm
Traitement du site 4/2260: La baie de Somme8.170Une destination de prédilection pour les amoureux de la nature
URL corrigée: https://www.france-voyage.com/tourisme/baie-somme-987.htm
Traitement du site 5/2260: Amiens8.063La capitale de la Picardie est célèbre pour sa cathédrale gothique
URL corrigée: https://www.france-voyage.com/tourisme/amiens-985.htm
T

# Cleaning Df

In [50]:
import re

# Fonction pour nettoyer le nom
def clean_name(name):
    # Trouve l'index du premier chiffre
    match = re.search(r'\d', name)
    if match:
        # Retourne le texte jusqu'au premier chiffre, en enlevant les espaces à la fin
        return name[:match.start()].strip()
    return name.strip()

# Appliquer la fonction à la colonne 'name'
df['name'] = df['name'].apply(clean_name)

# Afficher le résultat
print("Aperçu des noms nettoyés:")
df.head()

Aperçu des noms nettoyés:


,name,url,text_info
0,Le château de Pierrefonds,https://www.france-voyage.comchateau-pierrefon...,"À l'orée de laforêt de Compiègnedans l'Oise, l..."
1,Gerberoy,https://www.france-voyage.comgerberoy-571.htm,"Amoureux des fleurs et des roses, bienvenue à ..."
2,La Côte d'Opale,https://www.france-voyage.comcote-opale-626.htm,DeBray-Dunesdans le Nord jusqu'à Berck-sur-Mer...
3,La baie de Somme,https://www.france-voyage.combaie-somme-987.htm,"Plus grand estuaire du nord de la France, avec..."
4,Amiens,https://www.france-voyage.comamiens-985.htm,Capitale de la Picardie et ville d'Art et d'hi...


In [51]:
# Renommer le DataFrame
df_final_Site_Touristique = df

# Sauvegarder au format CSV
df_final_Site_Touristique.to_csv('df_final_Site_Touristique.csv', index=False, encoding='utf-8')

Aperçu du DataFrame final:
                        name  \
0  Le château de Pierrefonds   
1                   Gerberoy   
2            La Côte d'Opale   
3           La baie de Somme   
4                     Amiens   

                                                 url  \
0  https://www.france-voyage.comchateau-pierrefon...   
1      https://www.france-voyage.comgerberoy-571.htm   
2    https://www.france-voyage.comcote-opale-626.htm   
3    https://www.france-voyage.combaie-somme-987.htm   
4        https://www.france-voyage.comamiens-985.htm   

                                           text_info  
0  À l'orée de laforêt de Compiègnedans l'Oise, l...  
1  Amoureux des fleurs et des roses, bienvenue à ...  
2  DeBray-Dunesdans le Nord jusqu'à Berck-sur-Mer...  
3  Plus grand estuaire du nord de la France, avec...  
4  Capitale de la Picardie et ville d'Art et d'hi...  

Nombre total de sites: 2260
